# Xin Chen's implementation of the MBE-NN network

* Author: Xin Chen
* Email: Bismarrck@me.com

This jupyter notebook is used to repeat the work of http://doi.org/10.1021/acs.jctc.6b00994. 

The test cluster is $B_{20}$. The reference energies are calculated with CP2K-2.4 at tne PBE level using DZVP basis and GTH potentials.

## Overview

The structure of the deep convolutional neural network for the $Pt_{13}$ cluster is as follows:

<img src="./convnet.jpg" alt="Drawing" style="width: 600px;"/>

The input features are transformed interatomic distances. The output node represents the estimated DFT energies.
The detailed explanantion of this convolutionary neural network will be given in the following section **Inference**.

## Imports and Declarations

In this section we shall import python modules and declare global constants.

In [3]:
import tensorflow as tf
import numpy as np

In [6]:
# The patch size is always 1x1
PATCH_SIZE = 1

# The HDF5 database file.
HDF5_DATABASE_FILE = "c9h7n.hdf5"

# The dimension of the cluster.
NUM_SITES = 20

# The total number of structures in the XYZ file should be 209660.
XYZ_FILE = "c9h7n.xyz"
TOTAL_SIZE = 100

# Setup the dataset partitions.
TEST_SIZE = 10
VALID_SIZE = 10
TRAIN_SIZE = 60
LOAD_SIZE = TEST_SIZE + VALID_SIZE + TRAIN_SIZE

# Setup the random seed.
SEED = 235

# Setup the precision of floats. 
tf_type = tf.float32
np_type = np.float32

## Inference

This convnet is a slightly different from normal convolutionary neural networks. Suppose N is the batch size:

* The shape of the input tensor is: $[N, 1, C_{N}^{k}, C_{k}^{2}]$
* The **k** defines the many-body expansion. In this paper, k is selected to be 4.
* The patch (kernel) is always 1x1.
* The padding scheme is 'SAME' and the strides are 1 in both directions.
* A shape transpose action should be taken between layer 4 and 5.


It's a good habit to setup the scope name for every tensor. So we define a helper function here.

In [7]:
def variable_summaries(tensor):
  """
  Attach a lot of summaries to a Tensor (for TensorBoard visualization).

  Args:
    tensor: a Tensor.

  """
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(tensor)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tensor, mean))))
      tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(tensor))
    tf.summary.scalar('min', tf.reduce_min(tensor))
    tf.summary.histogram('histogram', tensor)

## Preparing the Dataset